Code assumes that data is in same directory as capstone_repo directory:  
```
capstone_repo  
│   README.md  
│   gsc_mfcc_extraction.ipynb    
│  
speech_commands_v0.02  
└───backward  
│   │   file01.wav  
│   │   file02.wav (etc)  
└───bed  
│   |..etc  
```

Version 2: MFCCs will be 20x125 and we will not calculate MFCCs for rejection class yet, just 20 keywords

In [1]:
#import libraries
import numpy as np
import os
import librosa
import time
import pandas as pd

In [2]:
data_path = '../speech_commands_quantized/'

In [3]:
# get list of word directories in speech commands dataset
list(os.walk(data_path))[0]

('../speech_commands_quantized/',
 ['backward',
  'bed',
  'bird',
  'cat',
  'dog',
  'down',
  'eight',
  'five',
  'follow',
  'forward',
  'four',
  'go',
  'happy',
  'house',
  'learn',
  'left',
  'marvin',
  'nine',
  'no',
  'off',
  'on',
  'one',
  'right',
  'seven',
  'sheila',
  'six',
  'stop',
  'three',
  'tree',
  'two',
  'up',
  'visual',
  'wow',
  'yes',
  'zero'],
 ['testing_list.txt', 'validation_list.txt'])

In [4]:
all_words = ['backward',
  'bed',
  'bird',
  'cat',
  'dog',
  'down',
  'eight',
  'five',
  'follow',
  'forward',
  'four',
  'go',
  'happy',
  'house',
  'learn',
  'left',
  'marvin',
  'nine',
  'no',
  'off',
  'on',
  'one',
  'right',
  'seven',
  'sheila',
  'six',
  'stop',
  'three',
  'tree',
  'two',
  'up',
  'visual',
  'wow',
  'yes',
  'zero']

In [5]:
#only these 20 words will act as keywords
word_list = ['up','down','left', 'right', 'stop', 'go', 'yes', 'no', 'on', 'off', 'one', 'two', 
             'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'zero']
len(word_list)

20

In [6]:
#the other 15 words will be combined into one class of rejection words
rejection_words = [i for i in all_words if i not in word_list]
print(rejection_words)
len(rejection_words)

['backward', 'bed', 'bird', 'cat', 'dog', 'follow', 'forward', 'happy', 'house', 'learn', 'marvin', 'sheila', 'tree', 'visual', 'wow']


15

In [7]:
#create base MFCC directory
base_mfcc = 'mfccs_quantized'
if not os.path.exists(base_mfcc):
    os.makedirs(base_mfcc)

In [8]:
#create training, validation, and test directories
if not os.path.exists(os.path.join(base_mfcc, 'training')):
    os.makedirs(os.path.join(base_mfcc, 'training'))
    
if not os.path.exists(os.path.join(base_mfcc, 'validation')):
    os.makedirs(os.path.join(base_mfcc, 'validation'))
    
if not os.path.exists(os.path.join(base_mfcc, 'test')):
    os.makedirs(os.path.join(base_mfcc, 'test'))

In [9]:
#create subdirectories that will contain MFCCs for each word
for directory in list(os.walk(base_mfcc))[0][1]:
    for i in all_words:
#         word_dir = base_mfcc + directory + '/' + i
        word_dir = os.path.join(base_mfcc, directory, i)
        if not os.path.exists(word_dir):
            os.makedirs(word_dir)

In [10]:
#read in list of validation files
val_path = data_path + 'validation_list.txt'
val_list = np.loadtxt(val_path, dtype = 'str')
val_list = [os.path.splitext(x)[0] for x in val_list]
display(val_list[:5])
print('Number of validation samples:', len(val_list))

['right/a69b9b3e_nohash_0',
 'right/439c84f4_nohash_1',
 'right/409c962a_nohash_1',
 'right/dbaf8fc6_nohash_2',
 'right/a6d586b7_nohash_1']

Number of validation samples: 9981


In [11]:
#read in list of test files
test_path = data_path + 'testing_list.txt'
test_list = np.loadtxt(test_path, dtype = 'str')
test_list = [os.path.splitext(x)[0] for x in test_list]
display(test_list[:5])
print('Number of testing samples:', len(test_list))

['right/bb05582b_nohash_3',
 'right/97f4c236_nohash_2',
 'right/f2e59fea_nohash_3',
 'right/fdb5155e_nohash_2',
 'right/dc75148d_nohash_0']

Number of testing samples: 11005


In [12]:
#set params for mfcc calculation
n_fft = 256
hop_length = 128

#loop through all files, calculate MFCCs, save them to the appropriate directory
start = time.time()

#GSC is sampled at 16000Hz
sample_rate = 16000

for i in all_words:
    #move to the directory for the ith word
    working_path = os.path.join(data_path, i)
    print(i)
    
    #loop through each file in the word's directory
    for j in os.listdir(working_path):
        #define paths we'll need
        word_path = i + '/' + j
#         word_path = os.path.join(i, j)
        file_path = data_path + word_path
        

        #load audio file
#         y, sr = librosa.load(file_path, sr = sample_rate)
        y = np.load(file_path)
    
        #pad shorter audio clips that don't have 16000 data points
        if (len(y) < sample_rate):
            y = np.pad(y, (0, (sample_rate - len(y))))

        #calculate MFCC
        mfcc = librosa.feature.mfcc(y=y, sr=sample_rate, n_fft = n_fft, hop_length = hop_length)

        #define save location directory
        word_dir = i + '_mfcc'

        #set save path to either train, test, or validation
        if os.path.splitext(word_path)[0] in test_list:
            save_path = os.path.join(base_mfcc, 'test', i, os.path.splitext(j)[0])
        elif os.path.splitext(word_path)[0] in val_list:
            save_path = os.path.join(base_mfcc, 'validation', i, os.path.splitext(j)[0])
        else:
            save_path = os.path.join(base_mfcc, 'training', i, os.path.splitext(j)[0])

        #save MFCC
        np.save(save_path, mfcc)

end = time.time()
print(end - start)

backward


C:\Users\dylaa\anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


bed
bird
cat
dog
down
eight
five
follow
forward
four
go
happy
house
learn
left
marvin
nine
no
off
on
one
right
seven
sheila
six
stop
three
tree
two
up
visual
wow
yes
zero
1929.8993742465973


In [13]:
os.path.splitext(j)[0]

'fffcabd1_nohash_0'

In [15]:
#let's make sure there are the right number of MFCCs in validation, test, and train
val_count = 0
test_count = 0
train_count = 0
for i in range(1,36):
    val_count += len(list(os.walk('mfccs_quantized/validation'))[i][2])
    test_count += len(list(os.walk('mfccs_quantized/test'))[i][2]) 
    train_count += len(list(os.walk('mfccs_quantized/training'))[i][2])
print(val_count, 'MFCCs in validation folder')
print(test_count, 'MFCCs in testing folder')
print(train_count, 'MFCCs in training folder')

9981 MFCCs in validation folder
11005 MFCCs in testing folder
84843 MFCCs in training folder
